In [0]:
pip install reverse_geocoder

In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta
     

In [0]:
input_path="/Volumes/earth_data/bronze/operationaldata/rig_count/"
# Load the JSON data into a Spark DataFrame
baker_rig_count_df = spark.read \
.option('header', True) \
.csv(input_path)

In [0]:

# Reshape earthquake data
baker_rig_count_selected = (
    baker_rig_count_df
    .select(
       
        col('Country').alias('country'),
        col('County').alias('county'),
        col('Basin').alias('basin'),
        col('DrillFor').alias('drill_for'),
        col('State/Province').alias('state_or_province'),
        col('Location').alias('location'),
        col('Trajectory').alias('trajectory'),
        col('US_PublishDate').alias('date'),
        col('Rig Count Value').cast('int').alias('rig_count'),
    )
)
    

In [0]:

# Convert 'time' and 'updated' to timestamp from Unix time
baker_rig_count_selected = (
    baker_rig_count_selected
    .withColumn('timestamp', to_timestamp('date', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))
    .withColumn('date', to_date('timestamp'))
)

In [0]:
silver_path="earth_data.silver.baker_rig_count"
baker_rig_count_selected.write.mode("overwrite").format("delta").saveAsTable(silver_path)